In [1]:
import gzip
import random
import matplotlib.pyplot as plt
import string
import numpy
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import defaultdict

### Purchase

#### Splitting data

In [3]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [4]:
data = []
for l in readGz("train.json.gz"):
    data.append(l)

In [117]:
train, validation, y_train, y_validation = train_test_split(data, [1]*len(data), test_size=0.5)

In [221]:
train = data

In [222]:
items = defaultdict(list)
for v in train:
    itemCat = v['categoryID']
    itemID = v['itemID']
    if items[itemID] == []:
        items[itemID] = [itemCat, 1]
    else:
        items[itemID][1] += 1

In [223]:
users = defaultdict(list)
for v in train:
    userID = v['reviewerID']
    itemCat = v['categoryID']
    if users[userID] == []:
        users[userID] = [1, [0,0,0,0,0]]
    else:
        users[userID][0] += 1
    users[userID][1][itemCat] += 1

In [224]:
mostCommonItems = []
for v in items:
    mostCommonItems.append([v, items[v][1]/len(train)])
    
mostCommonItems = sorted(mostCommonItems, key= lambda x:-x[1])
mostCommonItems = [v[0] for v in mostCommonItems]

In [225]:
notPurchased = []
pairs = []
count = 0
while count < 200000:
    user = random.randint(0, 199999)
    product = random.randint(0, 199999)
    v = {'reviewerID' : 0,
        'itemID' : 0}
    v['reviewerID'] = data[user]['reviewerID']
    v['itemID'] = data[product]['itemID']
    #if [v['reviewerID'], v['itemID']] not in pairs:
    notPurchased.append(v)
        #pairs.append([v['reviewerID'], v['itemID']])
    count += 1

In [226]:
train = data + notPurchased

In [227]:
y_train = [1]*len(data) + [0]*len(notPurchased) 

In [122]:
train = train + notPurchased[:100000]
validation = validation + notPurchased[100000:]

In [123]:
y_train = y_train + [0]*100000
y_validation = y_validation + [0]*100000

In [228]:
X_train = []
for v in train:
    itemID = v['itemID']
    userID = v['reviewerID']
    if itemID in items:
        nPurchases = items[itemID][1]
        itemCat = items[itemID][0]
        isPopular = int(mostCommonItems.index(itemID) < len(mostCommonItems)//2)
    else:
        nPurchases = 0
        itemCat = 0
        isPopular = 0
    if userID in users:
        userPurchases = users[userID][0]
        userCat = users[userID][1].index(max(users[userID][1]))
    else:
        userPurchases = 0
        userCat = 0
    X_train.append([1, nPurchases, itemCat, userPurchases, isPopular, userCat])

In [229]:
X_validation = []

    
for v in validation:
    itemID = v['itemID']
    userID = v['reviewerID']
    if itemID in items:
        nPurchases = items[itemID][1]
        itemCat = items[itemID][0]
        isPopular = int(mostCommonItems.index(itemID) < len(mostCommonItems)//2)
    else:
        nPurchases = 0
        itemCat = 0
        isPopular = 0
        
    if userID in users:
        userPurchases = users[userID][0]
        userCat = users[userID][1].index(max(users[userID][1]))
    else:
        userPurchases = 0
        userCat = 0
    X_validation.append([1,nPurchases,itemCat, userPurchases, isPopular, userCat])

In [111]:
svms = []
for c in [0.01]:
    clf = svm.LinearSVC(C=c)
    clf.fit(X_train, y_train)
    train_predictions = clf.predict(X_train) 
    #validation_predictions = clf.predict(X_validation)
    trainAcc = accuracy_score(y_train, train_predictions)
    #validationAcc = accuracy_score(y_validation, validation_predictions)
    print('C = ' + str(c) + ':')
    print('Train: ' + str(trainAcc))
    #print('Validation: ' + str(validationAcc))
    print()
    svms.append(clf)

C = 0.01:
Train: 0.5277675



#### Getting Users and Items features

In [112]:
X_test = []
for l in open("pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        #header
        #predictions.write(l)
        continue
    u,i = l.strip().split('-')

    itemID = i
    userID = u
    if itemID in items:
        nPurchases = items[itemID][1]
        itemCat = items[itemID][0]
        #isPopular = int(mostCommonItems.index(itemID) < len(mostCommonItems)//5)
    else:
        nPurchases = 0
        itemCat = 0
        isPopular = 0
    if userID in users:
        userPurchases = users[userID]
    else:
        userPurchases = 0
    X_test.append([nPurchases, userPurchases])

In [113]:
y_test = clf.predict(X_test)

In [115]:
len([x for x in y_test if x == 0])

6196

In [116]:
index = 0
predictions = open("predictions_Purchase.txt", 'w')
for l in open("pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')

    predictions.write(u + '-' + i + "," + str(y_test[index]) + "\n")
    index += 1
predictions.close()

#### Linear Regression

In [230]:
theta,residuals,rank,s = numpy.linalg.lstsq(X_train, y_train)

D:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [231]:
theta

array([ 5.00523905e-01, -1.01353888e-05, -5.64223045e-05,  6.31537726e-05,
       -6.96583821e-04, -6.62998595e-04])

In [232]:
train_predictions = []
for v in X_train:
    x = sum(v*theta)
    if x > 0.5:
        x = 1
    else:
        x = 0
    train_predictions.append(x)
trainAcc = accuracy_score(y_train, train_predictions)



In [233]:
validation_predictions = []
for v in X_validation:
    x = sum(v*theta)
    if x > 0.5:
        x = 1
    else:
        x = 0
    validation_predictions.append(x)
validationAcc = accuracy_score(y_validation, validation_predictions)

In [234]:
print('Train: ' + str(trainAcc))
print('Validation: ' + str(validationAcc))

Train: 0.50357
Validation: 0.50362


In [235]:
X_test = []
for l in open("pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        #header
        #predictions.write(l)
        continue
    u,i = l.strip().split('-')

    itemID = i
    userID = u
    if itemID in items:
        nPurchases = items[itemID][1]
        itemCat = items[itemID][0]
        isPopular = int(mostCommonItems.index(itemID) < len(mostCommonItems)//2)
    else:
        nPurchases = 0
        itemCat = 0
        isPopular = 0
    if userID in users:
        userPurchases = users[userID][0]
        userCat = users[userID][1].index(max(users[userID][1]))
    else:
        userPurchases = 0
        userCat = 0
    X_test.append([1,nPurchases,itemCat, userPurchases, isPopular, userCat])

In [236]:
y_test = []
for v in X_test:
    x = sum(v*theta)
    if x > 0.5:
        x = 1
    else:
        x = 0
    y_test.append(x)

In [238]:
len([x for x in y_test if x == 0])

13078

In [239]:
index = 0
predictions = open("predictions_Purchase.txt", 'w')
for l in open("pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')

    predictions.write(u + '-' + i + "," + str(y_test[index]) + "\n")
    index += 1
predictions.close()